# How precise must be the input velocity parameter ?

The input velocity (set through the parameter ``pos_cov``) must be given with a good precision. The error should not generally be larger than a spectrum channel (since the FWHM is around 1.5 channels)

Here's some ways you can determine the size of a channel.

In [1]:
from orcs.process import SpectralCube
# load spectral cube
cube = SpectralCube('/home/thomas/M31_SN3.merged.cm1.1.0.hdf5')

## Compute the channel width (in cm-1)

In [3]:
# you can simply get it from the axis of the cube (which is in cm-1)
print('channel width (cm-1)', cube.params.base_axis[1] - cube.params.base_axis[0])

channel width (cm-1) 2.0983247457061225


In [4]:
# you can relate the channel size to the line width as the FWHM of a sinc line is equal to 1.20671 x width 
# and the width of a sinc line is equal to 1.25 the channel size by construction (because there are 25% more 
# steps made than necessary on the left size of the ZPD to be able to compute the phase)
# the number of steps really contributing to the resolution are the steps made at the right of the ZPD which can be
# esaily calculated
print('line width (cm-1)', cube.params.line_fwhm / 1.20671)
print('number of steps at the right of ZPD (contributing to the resolution)', cube.params.step_nb - cube.params.zpd_index)
print('ratio of the step number / the number of steps at the right of the ZPD', cube.params.step_nb / float(cube.params.step_nb - cube.params.zpd_index))
print('channel width (cm-1)', cube.params.line_fwhm / 1.20671 / 1.25)

line width (cm-1) 2.6229059321332664
number of steps at the right of ZPD (contributing to the resolution) 672
ratio of the step number / the number of steps at the right of the ZPD 1.25
channel width (cm-1) 2.098324745706613


In [5]:
# Finally, you can relate the line width with the resolution at the mean wavenumber of the cube 
# and then compute the channel size
print('Spectral resolution: ', cube.params.resolution)
mean_cm1 = (cube.params.base_axis[-1] + cube.params.base_axis[0]) / 2
print('Mean wavenumber', mean_cm1)
line_fwhm = mean_cm1 / cube.params.resolution 
print('Line width (cm-1)',  line_fwhm)
print('channel width (cm-1)', line_fwhm / 1.20671 / 1.25)

Spectral resolution:  4733.531668752227
Mean wavenumber 14980.989521972366
Line width (cm-1) 3.1648651726294244
channel width (cm-1) 2.0981778041978103


## In terms of velocity

The channel width can be related to an uncertainty on the velocity of the lines at a given wavelength

In [6]:
import orb.utils.spectrum
channel_width = cube.params.base_axis[1] - cube.params.base_axis[0]
mean_cm1 = (cube.params.base_axis[-1] + cube.params.base_axis[0]) / 2
print('channel width in km/s at the mean wavenumber of the axis', orb.utils.spectrum.compute_radial_velocity(mean_cm1 + channel_width, mean_cm1, wavenumber=True))

channel width in km/s at the mean wavenumber of the axis -41.98773900973879


In [7]:
# it could be directly computed with the following formula
c = 3e5 # lightspeed in km/s
print('channel width in km/s at the mean wavenumber of the axis', c * channel_width / mean_cm1)

channel width in km/s at the mean wavenumber of the axis 42.019749282152794
